In [10]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk import word_tokenize, pos_tag, regexp_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import LogisticRegression
import random

In [2]:
train = pd.read_csv('train.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
train2 = train.copy()

In [4]:
# remove '\\n'
train2['comment_text'] = train2['comment_text'].map(lambda x: re.sub('\\n',' ',str(x)))
    
# remove any text starting with User... 
train2['comment_text'] = train2['comment_text'].map(lambda x: re.sub("\[\[User.*",'',str(x)))
    
# remove IP addresses or user IDs
train2['comment_text'] = train2['comment_text'].map(lambda x: re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",'',str(x)))
    
#remove email addresses 
train2['comment_text'] = train2['comment_text'].map(lambda x: re.sub("(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)",'',str(x)))

#remove WP::___
train2['comment_text'] = train2['comment_text'].map(lambda x: re.sub("WP:\w*",'',str(x)))

#remove all websites
train2['comment_text'] = train2['comment_text'].map(lambda x: re.sub("^(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?$",'',str(x)))



In [33]:
rng = random.randint(0,len(train))
origtestcomment = train['comment_text'][rng]
print(origtestcomment,'\n')
testcomment = train2['comment_text'][rng]
print(testcomment)

"

 Unblock Request 

A group of individuals, including at least one admin, has conspired to suppress relevant and topical information from being included in Wikipedia.  My attempt to thwart this organized censorship has resulted in blocking by User::Bbb23.  Someone that cares about censorship, please respond to this unblock request.  I am currently in the middle of a discussion on AfD and an ANI against another user, which this admin has stopped me from being able to participate in.  The ""system"" here is not working.  Please see " 

"   Unblock Request   A group of individuals, including at least one admin, has conspired to suppress relevant and topical information from being included in Wikipedia.  My attempt to thwart this organized censorship has resulted in blocking by User::Bbb23.  Someone that cares about censorship, please respond to this unblock request.  I am currently in the middle of a discussion on AfD and an ANI against another user, which this admin has stopped me from

In [11]:
snowball = SnowballStemmer('english')

In [50]:
tokens = [regexp_tokenize(x,pattern="([a-zA-Z]+(?:'[a-z]+)?)") for x in train2['comment_text']]
tokens[0]

['Explanation',
 'Why',
 'the',
 'edits',
 'made',
 'under',
 'my',
 'username',
 'Hardcore',
 'Metallica',
 'Fan',
 'were',
 'reverted',
 'They',
 "weren't",
 'vandalisms',
 'just',
 'closure',
 'on',
 'some',
 'GAs',
 'after',
 'I',
 'voted',
 'at',
 'New',
 'York',
 'Dolls',
 'FAC',
 'And',
 'please',
 "don't",
 'remove',
 'the',
 'template',
 'from',
 'the',
 'talk',
 'page',
 'since',
 "I'm",
 'retired',
 'now']

In [52]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\E\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [53]:
from nltk.corpus import words
stems_list = []
for list_of_tokens in tokens:
    stems = [snowball.stem(token) for token in list_of_tokens if token in words.words() == True]
    stems_list.append(stems)

KeyboardInterrupt: 

In [60]:
'fucker' in words.words()

False

In [43]:
stems_list[0]

['explan',
 'whi',
 'the',
 'edit',
 'made',
 'under',
 'my',
 'usernam',
 'hardcor',
 'metallica',
 'fan',
 'were',
 'revert',
 'they',
 'were',
 "n't",
 'vandal',
 'just',
 'closur',
 'on',
 'some',
 'gas',
 'after',
 'i',
 'vote',
 'at',
 'new',
 'york',
 'doll',
 'fac',
 'and',
 'pleas',
 'do',
 "n't",
 'remov',
 'the',
 'templat',
 'from',
 'the',
 'talk',
 'page',
 'sinc',
 'i',
 "'m",
 'retir',
 'now']

In [44]:
stems_tfidf = list(map(' '.join, stems_list))

In [46]:
tfidf = TfidfVectorizer(stop_words='english', token_pattern=r'\w{2,}')
tfidf.fit(stems_tfidf)
tfidf.get_feature_names()

["a'da",
 "a'edgf",
 "a'inggf",
 "a'innapropi",
 "a'la",
 "a'raf",
 "a'wa",
 "a'yaho",
 'aa',
 'aaa',
 'aaaa',
 'aaaaa',
 'aaaaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaani',
 'aaaaaaaaaah',
 'aaaaaaaaaahhhhhhhhhhhhhh',
 'aaaaaaaaadm',
 'aaaaaaaaaq',
 'aaaaaaaacfo',
 'aaaaaaaaczy',
 'aaaaaaaahhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh',
 'aaaaaaaayui',
 'aaaaaaahhhhhhhhhhhhhhhhhhhhhhhh',
 'aaaaaaw',
 'aaaaah',
 'aaaah',
 'aaaannnnyyyywwwwhhhheeeerrrreee',
 'aaaawwww',
 'aaaboyz',
 'aaag',
 'aaaghh',
 'aaah',
 'aaahhh',
 'aaai',
 'aaajad',
 'aaand',
 'aaarrrgggh',
 'aaaww',
 'aab',
 'aaba',
 'aaberg',
 'aabov',
 'aac',
 'aacargo',
 'aacd',
 'aachen',
 'aachi',
 'aad',
 'aademia',
 'aadil',
 'aadmi',
 'aadministr',
 'aadu',
 'aae',
 'aaets',
 'aaf',
 'aaffect',
 'aafia',
 'aaflight',
 'aafp',
 'aag',
 'aagadu',
 'aagf',
 'aagin',
 'aah',
 'aahahahahahaha',
 'aahank',
 'aahh',
 'aahil',
 'aahoa',
 'aai',
 'aaiha',
 'aajacksoniv',
 'aajonus',
 'aak',
 'aakash',
 '

## Test on multiple comments instead of one test comment

In [13]:
def lemmatize(comment):
    lems = [token.lemma_ for token in comment if (token.lemma_ != '-PRON-' and token.is_stop==False and 
                                   token.is_punct==False and token.is_digit==False and token.is_space==False 
                                          and not token.lemma_.startswith("'"))]
    return lems

In [14]:
test_bunch = no_spams_df['comment_text'].sample(1000)
test_bunch_nlp = map(nlp, test_bunch)
test_bunch_nlp_lem = map(lemmatize, test_bunch_nlp)
test_bunch_nlp_lem_joined = map(' '.join, test_bunch_nlp_lem)

In [16]:
tfid = TfidfVectorizer(stop_words='english', token_pattern="([a-zA-Z]+(?:'[a-z]+)?)")
tfid.fit(test_bunch_nlp_lem_joined)
tfid.get_feature_names()

['aa',
 'aadmi',
 'aafs',
 'aam',
 'aardvark',
 'abandon',
 'abate',
 'abc',
 'abdul',
 'aberration',
 'abet',
 'abhiramdasji',
 'abhor',
 'abi',
 'abide',
 'ability',
 'able',
 'aboriginal',
 'abortion',
 'abraham',
 'abroad',
 'absence',
 'absent',
 'absolutely',
 'absurd',
 'abuse',
 'abusive',
 'academic',
 'academy',
 'acc',
 'accept',
 'acceptable',
 'acceptance',
 'access',
 'accessory',
 'accident',
 'accidently',
 'accomplice',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accord',
 'accordance',
 'accordingly',
 'account',
 'accounts',
 'accuracy',
 'accurate',
 'accurately',
 'accusation',
 'accuse',
 'ace',
 'achaemenids',
 'ache',
 'achieve',
 'achievement',
 'acknowledge',
 'acount',
 'acquire',
 'acquis',
 'act',
 'action',
 'active',
 'activist',
 'activity',
 'acts',
 'actual',
 'actually',
 'actualy',
 'acually',
 'ad',
 'adam',
 'adams',
 'add',
 'adding',
 'addition',
 'additional',
 'additionally',
 'address',
 'adequately',
 'adhere',
 'adjacent',
 'adjecti